In [5]:
import numpy as np
import cvxpy as cp

# 1-қадам: Берілген мәліметтер
values = np.array([4, 2, 1, 7, 3, 6])  # Тауарлардың бағасы (пайдасы)
weights = np.array([5, 9, 8, 2, 6, 5])  # Тауарлардың салмағы
C = 15  # Сөмкенің максималды сыйымдылығы
n = len(values)  # Тауар саны

# 2-қадам: Сызықтық бағдарламалау есебін құру
c = values  # Максималдау үшін оң қалдырамыз
A = np.expand_dims(weights, 0)  # Векторды (1,6) өлшемді матрицаға түрлендіреміз
b = np.array([C])

# 3-қадам: Айнымалыны анықтау
x = cp.Variable(shape=n, integer=True)  # Бүтін санды айнымалы

# 4-қадам: Шектеулерді енгізу
constraint = [A @ x <= b, x >= 0, x <= 1]  # Жалпы салмақ шектеуі және 0 ≤ x ≤ 1 (бинарлы)

# 5-қадам: Мақсатты функцияны құру
objective = cp.Maximize(c @ x)  # Пайданың ең үлкен мәнін табу

# 6-қадам: Мәселені шешу
problem = cp.Problem(objective, constraints=constraint)
problem.solve()

# 7-қадам: Нәтижені шығару
print("Оптималды шешім:")
print("Таңдалған тауарлар саны:", np.round(x.value))  # Теріс мәндерден құтылу
print("Максималды пайда:", problem.value)


Оптималды шешім:
Таңдалған тауарлар саны: [ 1. -0. -0.  1.  0.  1.]
Максималды пайда: 17.0


In [1]:
#1
from scipy.optimize import linprog
import numpy as np

# Тасымалдау құны (у.е.)
c = [
    [2, 5, 3],  # A1 -> (B1, B2, B3)
    [7, 7, 6]   # A2 -> (B1, B2, B3)
]
c = np.array(c).flatten()

# Қоймадағы қорлар
a = [180, 220]

# Сауда орталықтарының қажеттіліктері
b = [110, 150, 140]

# Қоймадағы қор шектеулері (<= a)
A1 = np.array([
    [1, 1, 1, 0, 0, 0],  # A1
    [0, 0, 0, 1, 1, 1]   # A2
])
b1 = a

# Сауда орталықтарының қажеттіліктері (= b)
A2 = np.array([
    [1, 0, 0, 1, 0, 0],  # B1
    [0, 1, 0, 0, 1, 0],  # B2
    [0, 0, 1, 0, 0, 1]   # B3
])
b2 = b

# Сызықтық бағдарламалау есебін шешу
result = linprog(c, A_eq=np.vstack((A1, A2)), b_eq=np.hstack((b1, b2)), bounds=(0, None), method="highs")

# Нәтижені шығару
if result.success:
    print("Оптималды тасымалдау жоспары:")
    x = result.x.reshape(2, 3)  # Шешімді 2x3 матрицаға түрлендіру
    for i in range(2):
        for j in range(3):
            print(f"A{i+1} -> B{j+1}: {x[i, j]:.0f} дана")  # Әр қоймадан әр сауда орталығына жеткізу көлемі
    print(f"Жалпы шығындар: {result.fun:.2f} у.е.")  # Минималды шығындар
else:
    print("Шешім табылмады")


Оптималды тасымалдау жоспары:
A1 -> B1: 110 дана
A1 -> B2: 0 дана
A1 -> B3: 70 дана
A2 -> B1: 0 дана
A2 -> B2: 150 дана
A2 -> B3: 70 дана
Жалпы шығындар: 1900.00 у.е.


In [1]:
#2Мысал
import numpy as np
import cvxpy as cp

# 1-қадам: Берілген шығындар матрицасы (cij)
shygyn = np.array([
    [2, 3, 4],
    [1, 5, 2],
    [5, 2, 3]
])

n = shygyn.shape[0]  # Адамдар мен тапсырмалар саны (n x n)

# 2-қадам: Айнымалыларды анықтау (xij - қай адам қандай тапсырманы орындайтынын білдіреді)
tangdau = cp.Variable((n, n), boolean=True)

# 3-қадам: Минимизациялау мақсатты функциясы (жалпы шығын)
maksat = cp.sum(cp.multiply(shygyn, tangdau))

# 4-қадам: Шектеулерді орнату
shekteu = []

# Әр адам тек бір тапсырманы орындауы керек
for i in range(n):
    shekteu.append(cp.sum(tangdau[i, :]) == 1)

# Әр тапсырманы тек бір адам орындауы керек
for j in range(n):
    shekteu.append(cp.sum(tangdau[:, j]) == 1)

# 5-қадам: Мәселені шешу
masele = cp.Problem(cp.Minimize(maksat), shekteu)
masele.solve(solver=cp.GLPK_MI)

# 6-қадам: Нәтижені шығару
print("Minimaldy jalpy shygyn:", masele.value)
print("Tangdalgan tagayyndau:")
print(tangdau.value)


Minimaldy jalpy shygyn: 6.0
Tangdalgan tagayyndau:
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [10]:
#Тапсыпма 2
import numpy as np
from scipy.optimize import linear_sum_assignment

# ✅ Берілген шығындар матрицасы (әр орындаушыға арналған әр тапсырманың бағасы)
cost_matrix = np.array([
    [1000, 12, 10, 19, 8],
    [12, 1000, 3, 7, 2],
    [10, 3, 1000, 6, 20],
    [19, 7, 6, 1000, 4],
    [8, 2, 20, 4, 1000]
])

# ✅ Венгерлік әдісті қолдану арқылы оңтайлы тағайындауды табу
workers, tasks = linear_sum_assignment(cost_matrix)

# ✅ Орындаушы мен тапсырмаларды жұптау
assignments = list(zip(workers + 1, tasks + 1))  # +1, себебі индекстер 0-ден басталады

# ✅ Ең аз жалпы шығынды есептеу
min_cost = cost_matrix[workers, tasks].sum()

# ✅ Нәтижелерді шығару
print("Таңдалған тағайындаулар (Орындаушы -> Тапсырма):", assignments)
print("Минималды шығын:", min_cost)


Таңдалған тағайындаулар (Орындаушы -> Тапсырма): [(1, 5), (2, 4), (3, 2), (4, 3), (5, 1)]
Минималды шығын: 32


In [15]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus, value, PULP_CBC_CMD

# Нүктелер жиыны
nodes = ["A", "B", "C", "D", "E"]
edges = {
    ("A", "B"): 12, ("A", "C"): 10, ("A", "D"): 19, ("A", "E"): 8,
    ("B", "C"): 3, ("B", "D"): 7, ("B", "E"): 2,
    ("C", "D"): 6, ("C", "E"): 20,
    ("D", "E"): 4
}

# Айнымалыларды анықтау
x = LpVariable.dicts("x", edges, 0, 1, cat="Binary")

# Модель құру
model = LpProblem("TSP", LpMinimize)

# Мақсатты функция: Жол ұзындығын минимизациялау
model += lpSum(edges[i, j] * x[i, j] for i, j in edges), "Total Distance"

# Әр нүктеден бір шығу және бір кіру
for node in nodes:
    model += lpSum(x[i, j] for i, j in edges if i == node) >= 1, f"Out_{node}"
    model += lpSum(x[i, j] for i, j in edges if j == node) >= 1, f"In_{node}"


# Шеңберлік шектеу қою (MTZ әдісі арқылы)
u = LpVariable.dicts("u", nodes, 2, len(nodes), cat="Integer")
for i, j in edges:
    if i != "A" and j != "A":  
        model += u[i] - u[j] + (len(nodes) - 1) * x[i, j] <= len(nodes) - 2

# Модельді шешу
model.solve(PULP_CBC_CMD(msg=True))


# Нәтижелерді шығару
if LpStatus[model.status] == "Optimal":
    print("Ең қысқа жол табылды:")
    for (i, j) in edges:
        if value(x[i, j]) > 0.5:  # 1-ге жақын мәндерді тексеру
            print(f"{i} -> {j}")
    print("Ең қысқа жол ұзындығы:", value(model.objective))
else:
    print("Шешім табылмады.")


Шешім табылмады.


In [21]:
import numpy as np

def gradient_f(x, y):
    df_dx = 4*x - 4*y
    df_dy = -4*x + 4*y**3
    return np.array([df_dx, df_dy])

def gradient_descent(learning_rate=0.01, tolerance=1e-6, max_iters=10000):
    x, y = np.random.rand(2) * 10  # Бастапқы нүкте
    for i in range(max_iters):
        grad = gradient_f(x, y)
        x_new, y_new = x - learning_rate * grad[0], y - learning_rate * grad[1]
        
        if np.linalg.norm([x_new - x, y_new - y]) < tolerance:
            break
        x, y = x_new, y_new
    
    return x, y, i

x_min, y_min, iterations = gradient_descent()
print(f"Минимум нүктесі: (x, y) = ({x_min:.6f}, {y_min:.6f})")
print(f"Итерациялар саны: {iterations}")

Минимум нүктесі: (x, y) = (1.000039, 1.000016)
Итерациялар саны: 458


In [22]:
import numpy as np

# Функция
def f(x, y):
    return 2*x**2 - 4*x*y + y**4 + 2

# Градиенттер
def grad_f(x, y):
    df_dx = 4*x - 4*y
    df_dy = -4*x + 4*y**3
    return np.array([df_dx, df_dy])

# Бастапқы нүкте (анықталған аймақтан таңдау)
x, y = np.random.uniform(0.5, 1.5, 2)

# Параметрлер
alpha = 0.1  # Бастапқы оқу жылдамдығы
beta = 0.9    # Momentum коэффициенті
v_x, v_y = 0, 0
tol = 1e-6    # Тоқтату шарты
max_iter = 1000  # Максималды итерация саны

# Градиентті төмендеу алгоритмі
for iteration in range(max_iter):
    grad = grad_f(x, y)
    
    # Егер градиент өте аз болса, тоқтаймыз
    if np.linalg.norm(grad) < tol:
        break
    
    # Адаптивті learning rate
    alpha = 0.1 / (1 + 0.01 * iteration)
    
    # Momentum
    v_x = beta * v_x + (1 - beta) * grad[0]
    v_y = beta * v_y + (1 - beta) * grad[1]
    
    # Жаңарту
    x -= alpha * v_x
    y -= alpha * v_y

print(f"Минимум нүктесі: (x, y) = ({x:.6f}, {y:.6f})")
print(f"Итерациялар саны: {iteration}")


Минимум нүктесі: (x, y) = (1.000000, 1.000000)
Итерациялар саны: 234
